In [1]:
# mac475의 ipython 표준 style을 적용함
from IPython.core.display import HTML
styles = open("styles/custom.css", "r").read()
HTML( styles )

#6. Linear Model Selection and Regularization
* Linear model 확장을 시도한다
* 기존의 simple linear regression은 least square를 통해 fitting하였으나
* alternative fitting procedure 활용통해 다음 advantage를 확보가능
    - prediction accuracy
    - model interpretability
    

* Prediction accuracy
    0. 만약, true relationship이 대략 linear 형태라면 least square로도 충분히 low bias이며
    1. if n >> p : low variance
    2. if n << p : a lot of variability
    3. if p > n : least square 통해서는 coefficient estimation 불가 (variance는 infinite)
    * 이 경우, coefficients constain/ shrinkage 통해 variance reduce (단, bias의 negligible increase 발생가능)
* Model interpretability
    - multiple regression에 활용된 variables가 response와 관계성이 떨어지는 경우,
    - by removing variables 통해
    - model을 interpretable하게 만들 수 있음
    - 이를 feature selection, variable selection이라함


* 주요한 방법은 다음과 같음
    0. 기본 : $Y = \beta_0 + \beta_1X_1 + \cdots + \beta_pX_p + \epsilon$에서...
        
        $X_1, X_2, \cdots, X_p\,:\,predictors \quad \beta_0, \beta_1, \cdots, \beta_p\,:\, coefficients$
    1. Subset Selection
        - response와 related되는 p predictor의 subset의 identification후 least square fitting
        - 즉, predictor를 reduce하는 방법
    2. Shirinkage
        - all p predictor를 활용하나 estimated coefficient가 zero까지 shrinkage (regularization)가능
        - 즉, coefficient를 shrinking하는 방법 : 결과적으로 variable selection이 된다
    3. Demension Reduction
        - p predictor를 M-dimensional subspace로 projecting (단, M < p)
        - M different linear combinations, 혹은 projection computing 기반
        - 최종적으로 M projections가 least square 통해 predictor로 활용됨

##6.1 Subset Selection
* 수학적 기본 공식
    1. n개 원소를 가지는 집합에서 k개의 부분집합을 고르는 조합의 경우의 수 : 이항계수
        * 이항계수 (二項係數, 영어: binomial coefficient)는 주어진 크기의 (순서 없는) 조합의 가짓수이다.
        * 표현 : $$_nC_k = { n \choose k } = \frac{P(n,k)}{k!} = \frac{n!}{k! \cdot (n-k)!}$$
    
    2. n개 원소를 가지는 집합의 부분집합의 개수 : $2^n$

###6.1.1 Best Subset Selection
* Best subset selection위해, p predictors의 가능한 combination에 대한 least sqaure regression 수행필요. 즉,
    1. 1개 predictor를 가지는 model : p개
    2. 2개 predictor를 가지는 model 개수
        $$_pC_2 = { p \choose 2 } = \frac{p!}{2!(p-2)!} = \frac{p(p-1)(p-2)\cdots2\cdot1}{2!(p-2)\cdots2\cdot1} = \frac{p(p-1)}{2}$$
    
    3. 3개 predictor를 가지는 model 개수
        $$_pC_3 = { p \choose 3 } = \frac{p(p-1)(p-2)}{3!}$$
    
    4. 이후, 각 n개 predictor를 가지는 model들별로 best인 것을 확인가능함
    5. 이때, p개 predictor를 가지는 부분집합의 개수는 $2_p$개 이므로 computation 횟수가 지나치게 증가
    

* 따라서, 다음과 같은 Algorithm을 활용함
    1. $\mathcal{M}_0$ : null model (no predictor) : 단순 mean값을 가지는 model : 1개
    2. $k = 1, 2, \cdots p$ :
        - k개의 predictor를 가지는 model ${p \choose k}$개를 fitting : $\frac{p!}{k!(p-k)!}$개씩 fitting
        - (k개 predictor를 가지는 model중), best model을 $\mathcal{M}_k$라 함 : RSS값 최소, 혹은 $R^2$ 최고인 것 : p개 확보
    3. $\mathcal{M}_0, \cdots\mathcal{M}_p$중 cross-validation통해 $C_p (AIC), BIC, (adjusted)\:R^2$ 활용 best 선정 : p+1개중 선정
    
    
* 정리하면, 당초 $2^p$개 model에 대한 작업을 $p+1$개 model 대상으로 reduce함

###6.1.2 Stepwise Selection
* Best subset selection은 p의 증가에 따라 computation이 막대하고
* overfitting 및 high variance 가능성 있어
* 제한된 model에 대한 explore를 수행하는 stepwise metho를 대안으로 활용가능함

####Forward Stepwise Selection
* Best subset selection의 대안으로써 Forward stepwise selection : $2^p$ 개 model fitting 대신 훨씬 적은 model fitting
* 다음 Algorithm을 활용
    1. $\mathcal{M}_0$ : null model (no predictor)
    2. $k = 0, 1, \cdots p-1$ :
        - p-k개의 모든 model 대상으로 $\mathcal{M}_k$내의 predictor를 필수 고려(포함)
        - p-k개 model중 best 선정 ($RSS, R^2$ 기반)
    3. $\mathcal{M}_0, \cdots\mathcal{M}_p$중 cross-validation통해 $C_p (AIC), BIC, (adjusted)R^2$ 활용 best 선정
    
    
* Forward stepwise selection의 computational advantage는 명확하나, 최고의 model을 찾을 수 있다는 보장은 없음
$\because$ 반드시 앞에서 수행된 특정 predictor를 포함하고 있어야 함


####Backward Stepwise Selection
* Forward와 반대로 full p predictor에서 시작하여 iteratively하게 줄여나감

* 단, Backeward는 n > p에서만 수행가능. Forward는 n < p에서도 수행가능함


####Hybrid Approaches
* Forward와 같이 predictor를 stepwise하게 추가
* 추가후, 더이상 model fitting에 의미없는 predictor를 제거

###6.1.3 Choosing the Optimal Model
* Best subset selection, Forward selection, Backward selection 통해 결국 여러 model중 best model을 찾아내야 함
* 모든 predictors를 가지는 model이 최소 RSS와 최대 $R^2$값을 제공 : 이 값들이 training error와 관련되므로...
* Best model의 효과적인 선택방법이란... 결국 test error가 최소인 것임
* 하지만, training error는 test error에 대해 poor estimator임


* 따라서, test error가 낮은 best model을 찾기위해, 다음과 같은 두가지 접근방법이 존재
    1. indirectly test error estimation : training error adjustment 수행 ($\because$ training overfitting 대한 bias 해소)
    2. directly test error estimation : validation set/ cross-validation approach


####$C_p$, AIC, BIC, Adjusted $R^2$
* training MSE는 언제나 test MSE보다 낮음 ($\because$ training data를 활용하여 model fitting하므로)
* 따라서, 적절한 방법을 통해 - 즉, training error에 대한 $adjustment$통해 test error를 추정해봄
* 다음의 4가지 방법

|구분|내용|
|-|-|
|$C_p$|Mallow's $C_p$ : used to assess the fit of a regression model|
|AIC|Akaike information criterion|
|BIC|Bayesian information criterion|
|adjusted $R^2$|N/A|



1. Mallows's $C_p$
    $$C_p = \frac{1}{n}(RSS + 2d\hat{\sigma}^2)$$
    - 기본적으로, $RSS$에 $2d\hat{\sigma}^2$라는 penalty가 부여되는 것
    - $C_p$는 low test error인 model에 대해 작은 경향임

2. AIC : Akaike Information Criterion
    $$AIC = \frac{1}{n\hat{\sigma}^2}(RSS + 2d\hat{\sigma}^2) = \frac{1}{\hat{\sigma}^2}C_p 이다 (즉, C_p에\,proportional)$$

3. BIC : Bayesian Information Criterion
    $$BIC = \frac{1}{n}(RSS + log(n)d\hat{\sigma}^2)$$
    - 이때 $n > 7$경우, $\log{n} > 2$이므로, BIC값은 many variable시 penalty 무거워진다 ($C_p$, AIC에 비해)
    
4. adjusted $R^2$
    - 기본적으로 다음의 지식기반하에
    $$R^2 = 1 - \frac{RSS}{TSS}, \quad TSS = \sum(y_i - \bar{y})^2$$

    $$Adjusted \, R^2 = 1 - \frac{RSS/(n-d-1)}{TSS/(n-1)}$$
    - 이때, $RSS / (n-d-1)$값이 작아질수록 Adjusted $R^2$가 커짐
    - d값별로 $RSS / (n-d-1)$이 변동됨 (즉, noise d가 추가되면, RSS decrease가 매우 작음)
    * 단순 $R^2$와 달리, $adjusted\,R^2$는 *pays a price for the inclusion of unnecessary variables*된것


####Validation and Cross-Validation
* 과거의 경우 computing 문제로 인해 cross-validation 곤란, AIC, BIC, $C_p$, adjusted $R^2$가 충분히 의미있었음
* 하지만, cross-validation이 test error를 직접적으로 estimation 가능하므로 더 정확함

##6.2 Shrinkage Methods
* Subset selection은 subset predictor에 대한 least squares fitting 방법임
* Shrinkage는 all predictor에 대한 constrain, regularize 통해 shrink하는 방법임
    - variance를 reduce 가능 (*ridge regression, lasso*가 유명)

###6.2.1 Ridge Regression
* least squares fitting procedure시, $\beta_0, \beta_1, \cdots, \beta_p$의 estimation은 RSS 계산통해 수행
    $$RSS = \sum\limits_{i=1}^n\left(y_i = \beta_0 - \sum\limits_{j=1}^p\beta_jx_{ij}\right)^2$$

* Ridge regression은 least squares와 유사하나 다음과 같이 계산
    $$\sum\limits_{i=1}^n\left(y_i - \beta_0 - \sum\limits_{j=1}^p\beta_jx_{ij}\right)^2 + \lambda\sum\limits_{j=1}^p\beta_j^2 = RSS + \lambda\sum\limits_{j=1}^p\beta_j^2$$
    * 이때, $\lambda \ge 0$을 tuning parameter라 함
    * $\lambda\sum\limits_{j=1}^p\beta_j^2$를 shrinkage penalty라 함
    * $\lambda = 0$ : least square와 동일
    * $\lambda \to \infty$ : shrinkage penalty grows할수록, ridge regression coefficient estimate는 0으로 접근
    * least squares의 경우, coefficient set가 1개 존재하나, ridge regression의 경우 다양한 조합의 coefficient estimate 가능
    * 단, shrinkage penalty는 $\beta_1, \cdots, \beta_p$에 적용되며, $\beta_0$에는 적용되지 않음
        * 아래를 확인하면, $\lambda = 0 ~ \infty$ 증가시, coefficient가 0으로 shrink하는 것을 확인가능
    <img src="images/p216 figure 6.4.png" width = "65%"/>

####An Application to the Credit Data : 어려움

####Why Does Ridge Regression Improve Over Least Squares?
* Ridge regression이 least squares 대비 우수한 점은 *bias-variance trade-off에 기인*
    - $\lambda$ 증가시 flexibility 감소하여, bias는 증가하나 variance는 감소함 (bias : black, variance : green, MSE : purple)
    <img src="images/p218 figure 6.5.png" width = "65%"/>
* Ridge regression의 경우 p값이 증가하더라도, $\lambda$ 고정값 대상으로 single model fitting만 수행
* least squares 경우, p > n인경우 fitting 불가능하나, ridge의 경우 bias 증가를 감수 (variance 감소)하며 fitting 가능
* Ridge regression의 경우, least squares가 high variance인 경우 최적으로 수행가능

###6.2.2 The Lasso
* Ridge regression은 항상 all p predictor가 model에 포함되어야 하는 disadvantage
    - 이는 prediction accuracy는 문제없으나, model interpretation이 곤란
* Lasso는 이러한 Ridge regression의 대안
    $$\sum\limits_{i=1}^n\left(y_i - \beta_0 - \sum\limits_{j=1}^p\beta_jx_{ij}\right)^2 + \lambda\sum\limits_{j=1}^p\mid\beta_j\mid = RSS + \lambda\sum\limits_{j=1}^p\mid\beta_j\mid$$
    - 이때, *lasso*는 *ridge*와 다르게 $\mathcal{l}_1$ norm을 사용
    - lasso 역시 ridge와 같이 coefficient estimate을 0으로 shrink하지만, lasso의 $\mathcal{l}_1$은 일부 coefficient를 0으로 만들 수 있음 (단, $\lambda$가 매우 클 경우)
    - 따라서, subset selection과 같이 variable selection 수행이 가능함 (즉, interpret이 용이해짐)
    <img src="images/p220 figure 6.6.png" width = "80%"/>
* 위를 보면, $\lambda$ 증가에 따라 coefficient estimate이 0으로 수렴가능
* 우측의 그래프를 보면, 처음엔 Rating predictor만 개입되다가 이후 다른 predictor들이 추가로 출현가능
    - 하지만, ridge regression의 경우 언제나 all predictors가 model에 포함되어 fitting됨

####Another Formulation for Ridge Regression and the Lasso
* 어려움

####The Variable Selection Property of the Lasso
* 어려움

####Comparing the Lasso and Ridge Regression
* lasso는 ridge에 비해 simple하고 interpretable model을 만든다는 advantage 있음 (subset predictor 기반)
* 하지만, predict accuracy 측면에서 lasso와 ridge는 상호 보완적

* Lasso : 상대적으로 적은 predictor가 중요한 coefficient인 경우 유용
* Ridge : many predictor들에 기반한 function인 경우 유용

* Ridge/ Lasso 통해, least squares 지나치게 high variance 가진다면, bias 증가를 trade-off로 variance 낮출 수 있어 predict accuracy 향상가능하여, lasso의 경우 variable selection 또한 가능하므로 interpret 용이

* 요약 : Ridge, LASSO 모두 $\lambda$값 활용통해 coefficient의 flexibilty 낮춰 bias 증가시킴으로써, variance 낮출 수 있어 predict accuracy 향상모색한다 (이때, Ridge, Lasso의 predictor 전체, 일부 활용통한 장/단점 존재)

####A Simple Special Case for Ridge Regression and the Lasso
* 어려움

####Bayesian Interpretation for Ridge Regression and the Lasso
* 어려움

###6.2.3 Selecting the Tuning Parameter
* Ridge, Lasso의 경우 $\lambda$ parameter tuning위해 cross-validation 수행

##6.3 Dimension Reduction Methods
* 이제까지의 두가지 접근방법은 모두 variance를 조절하는 것으로써
    1. variable의 subset을 사용
    2. coefficient toward zero化
    이며, 두방법 모두 original predictors, $X_1, \cdots, X_p$를 변형없이 사용하는 기반
    
    * 이제는 predictor를 *transform*하는 방법인 **dimension reduction method**를 알아봄
    


* $M < p$인 경우, original p predictors에 대한 linear combinations를 $Z_1, Z_2, \cdots, Z_M$이라하면, 다음과 같이 표현
    $$Z_m = \sum\limits_{j=1}^p\phi_{jm}X_j$$
* 그렇다면, $\phi_{1m}, \phi_{2m} \cdots, \phi_{pm}$, $m = 1, \cdots, M$, 다음같은 linear regression model least squares 활용 fitting 가능
    $$y_i = \theta_0 + \sum\limits_{m=1}^M\theta_mz_{im} + \epsilon_i \quad (i = 1, \cdots, n)$$
* 이때, $\phi_{1m}, \phi_{2m} \cdots, \phi_{pm}$이 잘 선택되면, dimension reduction approach를 least squares를 outperform할 수 있음
* dimension reduction : $p+1$개의 coefficients, $\beta_0, \cdots, \beta_p$에 대한 estimation을 $M+1$개의 coefficients, $\theta_0, \theta_1, \cdots, \theta_M \quad (단, M < p)$로 간소화 한다는 의미임
* 즉, $p+1$ → $M+1$ problem으로 간소화
$$\sum\limits_{m=1}^M\theta_mz_{im} = \sum\limits_{m=1}^M\theta_m\sum\limits_{j=1}^p\phi_{jm}x_{ij} = \sum\limits_{j=1}^p\sum\limits_{m=1}^M\theta_m\phi_{jm}x_{ij} = \sum\limits_{j=1}^p\beta_jx_{ij} \quad (이때, \beta_j = \sum\limits_{m=1}^M\theta_m\phi_{jm})$$
* dimension reduction은 coefficient estimate 대상의 potential bias 가능성 있으나
    - $M \ll p$의 경우, variance를 줄여준다
    - $M = p$의 경우, least squares와 동일하다

* dimension reduction의 절차는
    * transformed predictors $Z_1, Z_2, \cdots, Z_M$을 확보
    * M개 predictor 대상 fitting
    * princial components, partial least square 방법이 있음

###6.3.1 Principal Components Regression
* [링크] http://darkpgmr.tistory.com/110 사전에 참고
* PCA : Principal components analysis는 large variables에서 low-dimension으로 추출/ 분석하는 방법
* unsupervised learning 수행시 자세하게 study
* 여기서는 regression을 위한 dimension reduction만 확인

####An Overview of Principal Components Analysis
* PCA는 $n \times p$ 배열의 dimension을 reducing하는 것
* first principal component direction은 observation vary가 max인 방향임 (즉, data의 분포중, 분산이 가장 큰 방향)
<img src="images/p230 figure 6.14.png" width = "70%"/>
* n개의 observation 존재시, $z_{i1}, \cdots z_{n1}$를 *principal component score*라 부름

####The Principal Components Regression Approach
* PCR (principal components regression) : $M$개 principal components $Z_1, \cdots Z_M$ 대해 linear regression 수행
* *directions in which $X_1, \cdots, X_p$ show the most variation are the direc-tions that are associated with Y*
* 아래의 좌측 : all predictors가 사용되는 경우, 우측 : 2개 predictor만이 사용되는 경우임
    - No of components가 증가함에 따라 : bias는 감소, variance는 증가, MSE는 U-shape을 보임
<img src="images/p230 figure 6.14.png" width = "70%"/>
* 단, PCR은 몇몇 components가 중요한 역할을 하고있는 경우에 좋은 해석력을 제공함
* PCR은 엄밀히 feature selection이 아니고, p개의 original feature에 대한 linear combination의 활용임
* 따라서, 의미적으로 보아 PCR은 (all predictors를 사용하는) ridge regression과 관련성이 큼
* PCR 수행시, 각 predictor에 대한 standardizing 필요

###6.3.2 Partial Least Squares
* PCR approach는 predictor들의 linear combination을 식별 수행 : unsupervised way임
* 따라서, predictor들을 가장 잘 설명하는 direction에 대한 보장은 할 수 없음
* PLC (Partial Least Squares)는 supervised way임 : feature approximation과 함께 response를 활용
    - 요약하면, PLS는 response와 predictor들 explain하는 direction을 찾고자 함
* M값은 cross-validation 통해 searching    

##6.4 Considerations in High Dimensions
###6.4.1 High-Dimensional Data
* 대부분의 classical problem들은 *low-dimensional*이며, 앞으로 *high-dimensional*을 study

###6.4.2 What Goes Wrong in High Dimensions?
* $p \gg n$인 경우, least squares는 n개 observations에 대해 perfect fitting을 해버려 overfitting 발생
    - 해석 : $p > n$, or $p \approx n$인 경우에 있어 simple least squares가 지나치게 flexible하므로 data에 overfitting
<img src="images/p240 figure 6.22.png" width = "70%"/>
    
* 기존의 training RSS, $R^2$을 adjusting하기 위한 $C_p, AIC, BIC$는 high dimensional에서 활용불가
    - $\because \hat{\sigma}^2$ = 0, $R^2 = 1$
    - 참고 : $C_p, AIC, BIC$ 모두 $\hat{\sigma}^2$에 대한 function이므로
    $$C_p = \frac{1}{n}(RSS + 2d\hat{\sigma}^2), \quad AIC = \frac{1}{n\hat{\sigma}^2}(RSS + 2d\hat{\sigma}^2), \quad BIC = \frac{1}{n}(RSS + log(n)d\hat{\sigma}^2)$$
    
###6.4.3 Regression in High Demensions
* forward stepwise, ridge regression, lasso, PCR은 주로 least squares에 비해 flexibility를 낮추는 approach임
    - 따라서, high-dimensional에 유용하게 활용가능
    - p = 20에선, 최소 regularization에서 test MSS가 최소
    - p = 50에선, 적절한 regularization을 통해 degrees of freedom이 20 근방일 때 test MSE가 최소
    - p = 2,000에선, regularization 수준이 의미없음
    <img src="images/p242 figure 6.24.png" width = "85%"/>
    

* 다음의 세가지 중요사항
    1. High-dimensional problem에 있어 regularization/ shrinkage는 중요한 역할
    2. 적절한 tuning parameter 선택은 performance에 중요
    3. test error는 dimensionality에 따라 증가
        

* *curse of dimensionality* : 무조건적인 feature의 증가가 model quality를 향상시키지는 않음
    - response에 관계성 떨어지는 noise성격 feature의 증가는 (nonzero coefficient로서 역할하게 되어) overfitting 유발가능

###6.4.4 Interpreting Results in High Dimensions
* High-dimensional에 있어 multicollinearity의 우려성 큼
* 더불어, high-dimensional에 있어 training data 대상의 각종 measure들은 신뢰곤란
* 즉, 반드시 test set을 대상으로 결과를 해석하도록, cross-validation을 활용하도록~